In [1]:
import torch
import numpy as np
import pandas as pd
import pickle
import os

In [2]:
print("CUDA present: ", torch.backends.cuda.is_built())
print("CUDA Api ver: ", torch.backends.cuda.sys.api_version)
print("CUDnn ver:", torch.backends.cudnn.version())
print("MKL available: ", torch.backends.mkl.is_available())

CUDA present:  True
CUDA Api ver:  1013
CUDnn ver: 7603
MKL available:  True


In [3]:
device = torch.device("cuda:0")
dtype = torch.double # =torch.float64; torch.floate=torch.float32; torch.half=torch.float16

In [4]:
# Detect the number of train data mini batches. Shuffle them and divide into a train/test parts.

fname_pattern = "./bengaliai-cv19/train_batch_{:03d}.pickle"
train_test_split = 0.7

mbatch_no = 0
while os.path.exists(fname_pattern.format(mbatch_no)):
    mbatch_no = mbatch_no + 1
print("{} train data file(s) found.".format(mbatch_no))

mbatch_train_no = round(mbatch_no * 0.7)
mbatch_test_no = mbatch_no - mbatch_train_no
mbatch_idx = np.random.permutation(mbatch_no)
mbatch_train_idx = mbatch_idx[:mbatch_train_no]
mbatch_test_idx = mbatch_idx[mbatch_train_no:]

197 train data file(s) found.


In [5]:
# pick up some mini-batch for autoencoder:
batchno = mbatch_train_idx[0]
batch = pickle.load(open("./bengaliai-cv19/train_batch_{:03d}".format(batchno) + ".pickle", "rb"))
X_batch = batch['X'][0:64, :, :].astype(np.float16) / 255.
Y_batch = batch['Y'][0:64, :].astype(np.float16) / 255. # X = Y, for an autoencoder

In [6]:
X = torch.from_numpy(X_batch.reshape((X_batch.shape[0], X_batch.shape[1] * X_batch.shape[2]))).cuda()
Y = X

In [7]:
class ConvDenseNNModel(torch.nn.Module):
    
    def __init__(self):
        super(ConvDenseNNModel, self).__init__()
        self.layers = []
        self.layers.append([
            lambda Inp: Inp.view(-1, 1, 137, 236)
        ])
        self.layers.append([
            torch.nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1).cuda().half(),
            torch.nn.LeakyReLU(0.2).cuda(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1).cuda()
        ])
        self.layers.append([
            torch.nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1).cuda().half(),
            torch.nn.LeakyReLU(0.2).cuda(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1).cuda()
        ])
        self.layers.append([
            torch.nn.Conv2d(32, 16, kernel_size=5, stride=1, padding=1).cuda().half(),
            torch.nn.LeakyReLU(0.2).cuda(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1).cuda()
        ])
        self.layers.append([
            torch.nn.Conv2d(16, 1, kernel_size=5, stride=1, padding=1).cuda().half(),
            torch.nn.LeakyReLU(0.2).cuda(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1).cuda(),
            lambda Inp: Inp.view(-1, 112)
        ])
        self.layers.append([
            torch.nn.Linear(112, 137*236).cuda().half(),
            torch.nn.LeakyReLU(0.2).cuda()
        ])
        self.layers.append([
            lambda Inp: Inp.view(-1, 137*236)
        ])
        self.model_modules = torch.nn.ModuleList()
        for layer in self.layers:
            for sublayer in layer:
                if isinstance(sublayer, torch.nn.Module):
                    self.model_modules.append(sublayer)
        
    def forward(self, x):
        for layer in self.layers:
            for sublayer in layer:
                x = sublayer(x)
        return x
    
    def parameters(self):
        return self.model_modules.parameters()
    

In [8]:
learning_rate = 0.001
iterations = 10

In [9]:
model = ConvDenseNNModel()
cost_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in range(iterations):
    Y_pred = model(X).cuda()
    loss = cost_fn(Y_pred, Y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print("Iteration #{}: loss={}".format(i + 1, loss.item()))

Iteration #1: loss=0.88720703125
Iteration #2: loss=nan
Iteration #3: loss=nan
Iteration #4: loss=nan
Iteration #5: loss=nan
Iteration #6: loss=nan
Iteration #7: loss=nan
Iteration #8: loss=nan
Iteration #9: loss=nan
Iteration #10: loss=nan
